In [3]:
import pandas as pd
import numpy as np
import re

In [2]:
notes = pd.read_csv('NOTEEVENTS.csv')
notes.groupby('CATEGORY').size()

/Users/zhangx/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


CATEGORY
Case Management         967
Consult                  98
Discharge summary     59652
ECG                  209051
Echo                  45794
General                8301
Nursing              223556
Nursing/other        822497
Nutrition              9418
Pharmacy                103
Physician            141624
Radiology            522279
Rehab Services         5431
Respiratory           31739
Social Work            2670
dtype: int64

In [9]:
notes.loc[0:5, ['ROW_ID', 'TEXT']]

,ROW_ID,TEXT
0,174,Admission Date: [**2151-7-16**] Dischar...
1,175,Admission Date: [**2118-6-2**] Discharg...
2,176,Admission Date: [**2119-5-4**] D...
3,177,Admission Date: [**2124-7-21**] ...
4,178,Admission Date: [**2162-3-3**] D...
5,179,Admission Date: [**2172-3-5**] D...


# Preprocess Radiology Reports

In [10]:
def _extract_final_report(text):
    sep = "______________________________________________________________________________"
    elements = text.split(sep)
    for i in np.arange(len(elements)):
        if 'FINAL REPORT' in elements[i]:
            final_report_section = elements[i].replace('FINAL REPORT', ' ')
            lb_removed = final_report_section.replace('\n', ' ').strip()
            pipe_removed = lb_removed.replace('|', ' ')
            square_removed = re.sub(r'\[\*\*[\w \-]+\*\*\]', '', pipe_removed)
            return square_removed
    return None

In [11]:
final_reports = [_extract_final_report(text) for text in notes.loc[notes.CATEGORY == 'Radiology', :].TEXT]
row_id = notes.loc[notes.CATEGORY == 'Radiology', :].reset_index().ROW_ID

In [12]:
radiology_reports = pd.DataFrame(data = {'ROW_ID': row_id, 'Radiology_final_report': final_reports})

save radiology reports for ClinPhen processing

In [ ]:
radiology_reports.dropna().to_csv('pre_processed_radiology_reports.csv', sep = '|', index=False)

In [ ]:
radiology_reports = pd.read_csv('pre_processed_radiology_reports.csv', sep = '|')

prepare a randomly selected subset for testing purposes

In [ ]:
N = len(radiology_reports)

In [ ]:
rand_indeces = np.random.choice(np.arange(N), 500)
subset = radiology_reports.iloc[rand_indeces, :]

In [ ]:
subset.to_csv('pre_processed_radiology_reports_test.csv', sep = '|', index = False)

In [ ]:
text = radiology_reports.loc[radiology_reports.ROW_ID == 738407, 'Radiology_final_report']
text.iloc[0]

# ClinPhen

One needs to download the ClinPhen source code and change a few lines that are not compatible with Python 3. Note PIP installed version will probably notwork with Python3. 

After than, run clinphen_bulk with the following settings:
```
python /home/zhangx/clinphen/clinphen_bulk --patcol ROW_ID --notecol Radiology_final_report --threads 32 /home/zhangx/text2hpo/pre_processed_radiology_reports.csv /home/zhangx/text2hpo/radiology_2_hpo.csv 
```

For MIMIC data (half million), they are processed on JAX cluster (so we can use 32 threads) in 4-5 hours. On a laptop, it probably will take > 8 hours. 